# Predicting Seizure Onset in Epileptic Patients Using Intracranial EEG Recordings

## Introduction

Work in progress

An [interesting article](http://cs229.stanford.edu/proj2014/Janet%20An,%20Amy%20Bearman,%20Catherine%20Dong,%20Predicting%20Seizure%20Onset%20in%20Epileptic%20Patients%20Using%20Intercranial%20EEG%20Recordings.pdf) to start working with. It hasn't many details on implementation, but gives some ideas of what to do.

## First steps

Load the data scientist weapons

In [ ]:
import itertools
import math
import os

from collections import Counter

import scipy.io
import scipy.stats as stats
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd

from IPython.core.debugger import Tracer
debug_here = Tracer()

Load a balanced set of Interictal (Non-Seizure, 0) and Preictal (Pre-Seizure, 1) examples.

In [ ]:
base_dir_train = u'/data/train_1/'
base_dir_tests = u'/data/test_1/'

INTERICTAL = 0
PREICTAL = 1

In [ ]:

def get_class_from_name(name):
    """
    Gets the class from the file name.
    
    The class is defined by the last number written in the file name.
    For example:
    
    Input: ".../1_1_1.mat"
    Output: 1.0
    
    Input: ".../1_1_0.mat"
    Output: 0.0
    """
    try:
        return float(name[-5])
    except:
        return 0.0

assert get_class_from_name('/train_1/1_1_0.mat') == 0.0
assert get_class_from_name('/train_1/1_1_1.mat') == 1.0


def get_file_names_and_classes(base_dir):
    ignored_files = ['.DS_Store', '1_45_1.mat']
    
    return np.array(
        [
            (file, get_class_from_name(file)) 
            for file in os.listdir(base_dir) if file not in ignored_files
        ],
        dtype=[('file', '|S16'), ('class', 'float32')]
    )

data_files_all = get_file_names_and_classes(base_dir_train)

# Count the occurrences of Interictal and Preictal classes
unique, counts = np.unique(data_files_all['class'], return_counts=True)
occurrences = dict(zip(unique, counts))

print('Interictal samples:', occurrences.get(INTERICTAL))
print('Preictal samples:', occurrences.get(PREICTAL))

In [ ]:
# If set to true, it will get a balanced set of Interictal and Preictal samples.
use_equal_size_sets = False

if use_equal_size_sets:
    set_size = 150

    # Randomly select an equal-size set of Interictal and Preictal samples
    data_random_interictal = np.random.choice(data_files_all[data_files_all['class'] == 0], size=set_size)
    data_random_preictal = np.random.choice(data_files_all[data_files_all['class'] == 1], size=set_size)

    # Merge the data sets and shufle the collection
    data_files = np.concatenate([data_random_interictal, data_random_preictal])
    data_files.dtype = data_files_all.dtype  # Sets the same dtype than the original collection
else:
    data_files = data_files_all

print('Data files shape:', data_files.shape, 'Data files size:', data_files.size)

### Loading data from .mat files and pre-process

Load data for each file and get the definitive training data.

Then create the `y` based on the classes of the files.
Note that because each file contains 16 channels, it is neccesary to
repeat each class in data_files['class'] 16 times, respecting the order of appearence.

In [ ]:
from numpy import linalg as LA

def perform_correlation(pd_data, corr_type):
    """
    Performs the correlation of type `corr_type` over the `pd_data`
    The parameter `pd_data` is a Pandas DataFrame
    """
    C = np.array(pd_data.corr(corr_type))

    # Sets all NaN and infinite values to 0
    C[np.isnan(C)] = 0
    C[np.isinf(C)] = 0

    # Calculates the Eigen values and vectors with the Numpy's Linear Algebra library
    w, v = LA.eig(C)

    # Sorts and casts all the values as real numbers
    w.sort()
    return w.real


def get_shannon_entropy(probs):
    """
    Given an array of probabilities of certain elements, returns the Shannon entropy
    """
    return -1 * np.sum(np.multiply(probs, np.log2(probs)), axis=0)

In [ ]:
import itertools
import math


def get_X_from_files(base_dir, files, show_progress=True):
    """
    Given a list of filenames, returns the final data we want to train the models.
    """
    X = []
    total_files = len(files)

    for i, filename in enumerate(files):
        if show_progress and i % int(total_files / 10) == 0:
            print(u'%{}: Loading file {}'.format(int(i * 100 / total_files), filename))

        try:
            mat_data = scipy.io.loadmat(''.join([base_dir, filename.decode('UTF-8')]))
        except ValueError as ex:
            print(u'Error loading MAT file {}: {}'.format(filename, str(ex)))
            continue

        # Gets a 16x240000 matrix => 16 channels reading data for 10 minutes at 400Hz
        egg_data = mat_data['dataStruct']['data'][0, 0]
        fs = mat_data['dataStruct']['iEEGsamplingRate'][0, 0][0, 0]
        
        # The shape of the EGG data
        n_time, n_channels = egg_data.shape
        
        # Define the window sample length. In this case is the number of meassurements in one minute
        samp_len = math.floor(fs * 60)
        samps_count = int(math.floor(n_time / samp_len))
        
        # The index of the samples in the whole EEG reading data
        samp_index = range(0, (samps_count + 1) * samp_len, samp_len)
        
        features = []
        
        for i in range(samps_count):
            epoch = egg_data[samp_index[i]: samp_index[i + 1], :]
            
            #
            # Compute Shannon's entropy, spectral edge and correlation matrix
            # segments corresponding to frequency bands
            #
            lvl = np.array([0.1, 4, 8, 12, 30, 70, 180])  # Frequency levels in Hz
            lseg = np.round(n_time / fs * lvl).astype('int')
            
            # Calculates the Fourier transformation under the period from 0 to n_time / fs * lvl[-1]
            D = np.absolute(np.fft.fft(epoch, n=lseg[-1], axis=0))
            D[0, :] = 0  # set the DC component to zero
            D /= D.sum()  # Normalize each channel
            
            dspect = np.zeros((len(lvl) - 1, n_channels))
            for j in range(len(dspect)):
                dspect[j, :] = 2 * np.sum(D[lseg[j]: lseg[j+1], :], axis=0)
            
            # Gets the shannon's entropy
            shannon_entropy = get_shannon_entropy(dspect)
            
            #
            # Find the spectral edge frequency
            #
            sfreq = fs
            tfreq = 40
            ppow = 0.5
            
            topfreq = int(round(n_time / sfreq * tfreq)) + 1
            A = np.cumsum(D[:topfreq, :])
            B = A - A.max() * ppow
            spedge = np.min(np.abs(B))
            spedge = (spedge - 1) / (topfreq - 1) * tfreq
            
            # Calculate correlation matrix and its eigenvalues (b/w channels)
            data = pd.DataFrame(data=epoch)
            pearson_lxchannels = perform_correlation(data, 'pearson')
            spearman_lxchannels = perform_correlation(data, 'spearman')

            # Calculate correlation matrix and its eigenvalues (b/w freq)
            data = pd.DataFrame(data=dspect)
            pearson_lxfreqbands = perform_correlation(data, 'pearson')
            spearman_lxfreqbands = perform_correlation(data, 'spearman')
            
            #
            # Spectral entropy for dyadic bands
            #

            # Find number of dyadic levels
            ldat = int(math.floor(n_time / 2.0))
            no_levels = int(math.floor(math.log(ldat, 2.0)))
            seg = math.floor(ldat / pow(2.0, no_levels - 1))

            # Find the power spectrum at each dyadic level
            dspect = np.zeros((no_levels, n_channels))
            for j in range(no_levels-1,-1,-1):
                dspect[j,:] = 2 * np.sum(D[int(math.floor(ldat / 2.0)) + 1: ldat, :], axis=0)
                ldat = int(math.floor(ldat / 2.0))

            # Find the Shannon's entropy
            shannon_entropy_dyd = get_shannon_entropy(dspect)

            # Find correlation between channels
            pearson_lxchannels_dyd = perform_correlation(pd.DataFrame(data=dspect), 'pearson')
            spearman_lxchannels_dyd = perform_correlation(pd.DataFrame(data=dspect), 'spearman')
            
            #
            # Hjorth parameters
            #

            # Activity
            activity = np.var(epoch, axis=0)

            # Mobility
            mobility = np.divide(
                np.std(np.diff(epoch, axis=0)), 
                np.std(epoch, axis=0)
            )

            #
            # Complexity
            # std of second derivative for each channel
            #
            complexity = np.divide(
                np.divide(
                    np.std(np.diff(np.diff(epoch, axis=0), axis=0), axis=0),
                    np.std(np.diff(epoch, axis=0), axis=0)
                ),
                mobility
            )
            
            #
            # Statistical properties
            #
            
            # Skewness
            sk = stats.skew(epoch)

            # Kurtosis
            kurt = stats.kurtosis(epoch)
            
            features = np.concatenate((
                features,
                shannon_entropy.ravel(),
                spedge.ravel(),
                pearson_lxchannels.ravel(),
                pearson_lxfreqbands.ravel(),
                spearman_lxchannels.ravel(),
                spearman_lxfreqbands.ravel(),
                shannon_entropy_dyd.ravel(),
                pearson_lxchannels_dyd.ravel(),
                spearman_lxchannels_dyd.ravel(),
                activity.ravel(),
                mobility.ravel(),
                complexity.ravel(),
                sk.ravel(),
                kurt.ravel(),
            ))
            
        X.append(features)

    X = np.array(X)
    X[np.isnan(X)] = 0
    X[np.isinf(X)] = 0
    return X.real

In [ ]:
%time X = get_X_from_files(base_dir_train, data_files['file'], show_progress=True)
y = data_files['class']

print('X shape:', X.shape, 'X size:', X.size)
print('y shape:', y.shape, 'y size:', y.size)

## Pre-processing

### Resamblation and normalization

In [ ]:
from sklearn.preprocessing import normalize

# Normalizes the data
normalize(X, copy=False)

# Plots a user normalized sample
matplotlib.rcParams['figure.figsize'] = (20.0, 5.0)
print('Showing case of file:', data_files['file'][0])
for i in range(16):
    plt.subplot(8, 2, i + 1)
    plt.plot(X[i])

# Model selection and predictions

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
for n, t in zip(['X_train', 'X_test', 'y_train', 'y_test'], [X_train, X_test, y_train, y_test]):
    print('{} shape'.format(n), t.shape, '{} size'.format(n), t.size)

In [ ]:
from sklearn import linear_model

results = {}

for C in [10, 100, 500, 750, 950, 1000, 1050, 1250, 1500, 10000]:
    print('Creating model with C={}'.format(C))

    clf = linear_model.LogisticRegression(C=C, n_jobs=4, solver='liblinear', verbose=10)

    %time clf.fit(X_train, y_train)

    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

    %time y_pred = clf.predict(X_test)

    results[C] = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1_score': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred),
    }
    
    for key, score in results[C].items():
        print('{}: {}'.format(key, score))
        
data = [(C, score['roc_auc']) for C, score in results.items()]

In [ ]:
print(data)
plt.plot([d[1] for d in sorted(data)])

# Results

60 samples (50/50)
-------------------

(C=.1, n_jobs=1, solver='liblinear')
```
CPU times: user 0 ns, sys: 10 ms, total: 10 ms
Wall time: 5.49 ms
Accuracy: 0.637223974763
Precision: 0.627218934911
Recall: 0.670886075949
F1 score: 0.648318042813
```

120 samples (50/50)
-------------------

(C=.1, n_jobs=1, solver='liblinear')

```
CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 6.98 ms
Accuracy: 0.61356466877
Precision: 0.611111111111
Recall: 0.597444089457
F1 score: 0.604200323102
```

(C=1, n_jobs=1, solver='liblinear')

```
CPU times: user 0 ns, sys: 20 ms, total: 20 ms
Wall time: 10.9 ms
Accuracy: 0.652996845426
Precision: 0.650485436893
Recall: 0.642172523962
F1 score: 0.646302250804
```

(C=10, n_jobs=1, solver='liblinear')

```
CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 7.13 ms
Accuracy: 0.708201892744
Precision: 0.703821656051
Recall: 0.70607028754
F1 score: 0.704944178628
```

238 samples
-----------

(C=16, n_jobs=1, solver='liblinear')

```
CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 21 ms
Accuracy: 0.695679796696
Precision: 0.684848484848
Recall: 0.720663265306
F1 score: 0.702299564947
```